# Capítulo 02: Instalação e Configuração

Guia completo de instalação, configuração e setup do DuckDB com extensões essenciais.

## 📦 Instalação de Dependências

In [ ]:
# Instalar DuckDB e extensões necessárias
%pip install duckdb>=0.10.3 deltalake pandas pyarrow -q

import duckdb
import sys

print(f"✓ DuckDB versão: {duckdb.__version__}")

# Verificar versão mínima para suporte Delta
if duckdb.__version__ < '0.10.3':
    print("⚠ DuckDB 0.10.3+ necessário para suporte Delta")
    sys.exit(1)
else:
    print("✓ Versão compatível com extensão Delta")

## 🔌 Configuração Básica de Conexão

Criação e configuração de conexões DuckDB (em memória e persistente).

In [ ]:
# Conexão em memória (rápida, não persistente)
con = duckdb.connect(':memory:')

# Teste básico
result = con.execute("SELECT 42 as answer, 'Hello DuckDB' as message").fetchall()
print(f"Teste de conexão: {result}")

# Conexão persistente (salva em disco)
# con_persistent = duckdb.connect('my_database.db')

# Conexão read-only
# con_readonly = duckdb.connect('my_database.db', read_only=True)

print("✓ Conexão estabelecida com sucesso")

## ⚙️ Configuração de Performance

Ajuste de memória e threads para otimizar performance.

In [ ]:
# Configurar limite de memória e threads
con.execute("SET memory_limit='4GB'")
con.execute("SET threads=4")

# Verificar configurações
config = con.execute("""
    SELECT name, value 
    FROM duckdb_settings() 
    WHERE name IN ('memory_limit', 'threads')
""").fetchdf()

print("Configurações de Performance:")
print(config)

## 🔧 Instalação de Extensões

Instalação e carregamento de extensões essenciais (delta, httpfs, parquet).

In [ ]:
import importlib.util

def safe_install_ext(con, ext_name):
    """Instala e carrega extensão de forma segura"""
    try:
        con.execute(f"INSTALL {ext_name}")
        con.execute(f"LOAD {ext_name}")
        print(f"✓ {ext_name} instalado e carregado")
        return True
    except Exception as e:
        print(f"⚠ Falha em {ext_name}: {e}")
        return False

# Instalar extensões necessárias
extensions = ['delta', 'httpfs', 'parquet']

for ext in extensions:
    safe_install_ext(con, ext)

# Verificar extensões carregadas
loaded = con.execute("""
    SELECT extension_name, loaded, installed
    FROM duckdb_extensions()
    WHERE extension_name IN ('delta', 'httpfs', 'parquet')
""").fetchdf()

print("\n📋 Status das Extensões:")
print(loaded)

## ✅ Teste de Funcionalidade Delta

Validação da extensão Delta com criação e leitura de tabela de teste.

In [ ]:
from deltalake import write_deltalake

# Criar DataFrame de teste
df_test = con.execute("""
    SELECT
        i as id,
        'value-' || i as value,
        i * 10.5 as amount
    FROM range(0, 10) tbl(i)
""").df()

# Escrever tabela Delta de teste
write_deltalake("./test_delta", df_test, mode="overwrite")
print("✓ Tabela Delta de teste criada")

# Ler com DuckDB
result = con.execute("""
    SELECT COUNT(*) as total, SUM(amount) as sum_amount
    FROM delta_scan('./test_delta')
""").fetchone()

print(f"✓ Teste Delta bem-sucedido! {result[0]} registros, soma: {result[1]}")

# Capitulo 02 Instalacao E Configuracao

Notebook gerado automaticamente a partir do código fonte python.
